# Comparison of chinese segmenters

I compare precision of two chinese segmenters:
 - [stanford nlp chinese segmenter](https://nlp.stanford.edu/software/segmenter.html)
 - [jieba](https://github.com/fxsjy/jieba) python package (java and rust implementation are floating around on github)
 
Segmentation was compared against this dataset:
https://www.microsoft.com/en-us/download/details.aspx?id=52531

Segmentation_distance between two segmentations is calculated int wo steps:
 - Convert segmented sentences into list of indices of word boundaries ("I am Groot" would be converted into `[1,3]`),
 - Calculate Levenstein distance between two lists of word boundaries. Levenstein distance is minimal number of insertions, deletions or substitutions needed to convert one list to another.

In [58]:
import jieba
import nltk
from lxml import etree
from collections import defaultdict
from utils import segmentation_distance

In [59]:
Parser=etree.XMLParser(encoding='gb18030')
tree=etree.parse(open('msra-chinese-word-segmentation-data-v1/msra_bakeoff3_training.xml',encoding='gb18030'),parser=Parser)

In [60]:
root=tree.getroot()
s=root.getchildren()[0]

sentences=[]
for s in root.getchildren():
    sent=[]
    for w in s.getchildren():
        wchildren=w.getchildren()
        word=[]
        if wchildren:
            for i in wchildren:
                if i.text:
                    word.append(i.text)
            sent.append(''.join(word))     #I assume that if the word has children, they should be joined together
        else:
            if w.text:
                sent.append(w.text)
        if len(wchildren)>1:
            print(sent)
    sentences.append(sent)
            

['我们', '变', '而', '以', '书会友', '，', '以', '书', '结缘', '，', '把', '欧美']
['我们', '变', '而', '以', '书会友', '，', '以', '书', '结缘', '，', '把', '欧美', '、', '港台']
['这', '两百余册', '革命', '历史', '文献', '完整', '地', '反映', '了', '从', '１９０８年', '到', '建国', '前', '马列']
['当年', '，', '淮海']
['当年', '，', '淮海', '的', '指挥', '者', '之一', '陈毅', '就', '曾经', '说', '过', '这样', '一', '句', '名言', '：', '“', '淮海']
['在', '淮海']
['四川牧区', '地处', '青藏']
['女篮', '运动', '这', '两年', '在', '欧美']
['在', '世锦赛', '之前', '参加', '的', '一系列', '热身赛', '中', '，', '中国队', '的', '主要', '对手', '是', '美国', '、', '澳大利亚', '、', '斯洛伐克', '等', '欧美']
['快攻', '曾经', '是', '中国队', '对付', '球风', '硬朗', '、', '体格', '强壮', '的', '欧美']
['由于', '训练', '的', '强度', '难以', '提高', '，', '面对', '体格', '强壮', '、', '作风', '硬朗', '的', '欧美']
['当年', '延安', '和', '南泥湾', '的', '八路军', '和', '干部', '，', '就', '是', '用', '它', '来', '照明', '纺线', '线', '、', '织布', '，', '用', '它', '来', '学习', '马列']
['记者', '参加', '解放军总政文化部', '最近', '组织', '的', '“', '全民', '健身', '走', '军营', '”', '采访', '，', '在', '陆海空']
['但', '他们', '却', '把', '这', '笔', '可观', '的', '稿费', '收入', '

['我', '见到', '他', '的', '时候', '，', '注意', '到', '他', '左', '胸', '前', '挂', '着', '４枚', '纪念章', '，', '分别', '是', '解放', '东北', '纪念章', '、', '华北', '解放', '纪念章', '、', '美朝']
['党', '的', '十五大', '高举', '邓小平', '理论', '伟大', '旗帜', '，', '明确', '把', '这', '一', '理论', '确立', '为', '党', '的', '指导思想', '并', '郑重', '写', '进', '党章', '，', '号召', '全党', '兴起', '一个', '学习', '马列']
['本届', '博览会', '由', '国家体育总局', '、', '福建省人民政府', '、', '中国体育用品联合会', '主办', '，', '届时', '大陆', '及', '港澳台']
['现在', '，', '“', '郑海霞', '时代', '”', '的', '鼎盛', '期', '已', '过', '，', '中国女篮', '面临', '着', '“', '向', '谁', '学习', '”', '的', '问题', '，', '是', '欧美']
['欧美']
['欧美']
['而', '９０年代中期', '以来', '，', '欧美']
['亚洲', '篮球', '一向', '以', '“', '快', '、', '灵', '、', '准', '”', '著称', '，', '要是', '没', '了', '这些', '长处', '，', '同', '身强体壮', '的', '欧美']
['据', '分析', '，', '韩国', '出口', '下降', '的', '原因', '主要', '有', '两个', '方面', '：', '一', '是', '韩国', '本身', '的', '金融', '系统', '尚未', '恢复', '正常', '运转', '，', '包括', '汽车', '和', '半导体', '在内', '的', '全部', '出口', '行业', '仍', '处于', '困难', '之中', '；', '二', '是', '欧美']
['但', '具有', '讽刺'

['我们', '相信', '，', '此次', '中美']
['我们', '相信', '，', '此次', '中美', '高层', '会晤', '将', '有利于', '进一步', '推动', '中美']
['我们', '相信', '，', '此次', '中美', '高层', '会晤', '将', '有利于', '进一步', '推动', '中美', '两国', '建立', '建设性', '的', '战略', '伙伴', '关系', '，', '这', '正是', '我们', '旅居', '美国', '的', '华', '华', '和', '中美']
['同', '一天', '，', '总部', '设', '在', '纽约', '的', '美国经贸科技总会', '在', '给', '克林顿', '和', '美国', '参众']
['同', '一天', '，', '总部', '设', '在', '纽约', '的', '美国经贸科技总会', '在', '给', '克林顿', '和', '美国', '参众', '领导人', '的', '公开信', '中', '说', '，', '克林顿', '此次', '去', '中国', '意义', '非常', '重大', '，', '这次', '访问', '的', '成功', '“', '将', '会', '把', '中美']
['事实', '证明', '，', '只有', '两国', '继续', '加深', '相互', '认识', '和', '理解', '，', '扩大', '共识', '，', '才', '能', '把', '一个', '稳定', '、', '健康', '的', '中美']
['中美']
['中美', '两国', '首脑', '经常', '会晤', '，', '表明', '中美']
['江泽民', '主席', '去年秋天', '访', '美', '以来', '，', '中美']
['关于', '中非']
['关于', '中非', '合作', '，', '唐', '外长', '说', '，', '他', '亲眼', '看到', '，', '中非']
['唐', '外长', '指出', '，', '中非']
['唐', '外长', '指出', '，', '中非', '友谊', '深深', '植根', '于', '人民',

['预计', '，', '３０日', '夜间', '到', '７月１日', '白天', '，', '黄', '淮', '、', '江淮']
['预计', '，', '３０日', '夜间', '到', '７月１日', '白天', '，', '黄', '淮', '、', '江淮', '、', '汉水', '流域', '、', '江南', '西部', '、', '华南', '大', '部', '和', '西南', '地区', '东部', '有', '小', '到', '中雨', '，', '其中', '黄', '淮', '、', '江淮']
['又', '讯', '今年年初', '即', '试', '运营', '的', '我国', '首家', '正式', '注册', '的', '中美']
['据', '国家发展计划委员会', '介绍', '，', '这是', '继', '去年', '江泽民', '主席', '访', '美', '期间', '两国', '政府', '签署', '的', '中美']
['据', '国家发展计划委员会', '介绍', '，', '这是', '继', '去年', '江泽民', '主席', '访', '美', '期间', '两国', '政府', '签署', '的', '中美', '和平', '利用', '核', '技术', '合作', '的', '意向', '性', '协议', '以及', '中美']
['据', '国家发展计划委员会', '介绍', '，', '这是', '继', '去年', '江泽民', '主席', '访', '美', '期间', '两国', '政府', '签署', '的', '中美', '和平', '利用', '核', '技术', '合作', '的', '意向', '性', '协议', '以及', '中美', '和平', '利用', '核能', '合作', '协定', '正式', '启动', '后', '的', '又', '一', '项', '重要', '协议', '，', '对', '推动', '中美']
['新华社', '北京', '６月２９日', '电', '中国国家发展计划委员会', '和', '美国能源部', '的', '代表', '今天', '在', '这里', '签署', '了', '中美']
['中美']
['至

['她', '信守', '质量', '第一', '、', '信誉', '第一', '的', '经营', '之', '道', '，', '一丝不苟', '地', '按', '工艺', '要求', '加工', '服装', '，', '赢得', '了', '良好', '的', '市场', '信誉', '，', '现在', '产品', '已', '远销', '欧美']
['早年', '辗转', '于', '闽', '粤', '港台']
['他', '尖锐', '地', '批评', '某些', '同志', '天天', '讲', '毛泽东', '，', '却', '忘记', '了', '从', '实际', '出发', '、', '理论', '联系实际', '的', '原则', '，', '实质上', '是', '主张', '只要', '照抄', '照搬', '马克思', '、', '列宁', '、', '毛泽东', '同志', '的', '原话', '就', '行', '了', '，', '要不然', '就', '说', '这是', '违反', '了', '马列']
['在', '２０年', '前', '这', '场', '关系', '党', '和', '国家', '前途', '命运', '的', '大', '讨论', '中', '，', '党', '的', '新闻', '工作者', '、', '理论', '工作者', '、', '科学', '工作者', '等', '相互', '支持', '，', '密切', '配合', '，', '在', '邓小平', '同志', '等', '老一辈', '革命家', '的', '领导', '支持', '下', '，', '顶住', '沉重', '的', '压力', '，', '以', '高度', '的', '社会', '责任感', '和', '极大', '的', '理论', '勇气', '，', '撰写', '文章', '，', '投入', '讨论', '，', '宣传', '检验', '真理', '的', '标准', '只能', '是', '社会实践', '，', '强调', '理论', '与', '实践', '的', '统一', '是', '马克思', '的', '一个', '最', '基本', '的', '原则', '，', '严肃'

In [61]:
len(sentences)

46364

Install stanfordCoreNlp as described here https://github.com/ellie-icekler/StanfordCoreNLP-Chinese/blob/master/README.md

Start service by:

java -Xmx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -serverProperties StanfordCoreNLP-chinese.properties -preload tokenize,ssplit,pos,lemma,ner,parse -status_port 9001  -port 9001 -timeout 15000

In [62]:
from nltk.tokenize.stanford import CoreNLPParser
sttok = CoreNLPParser('http://localhost:9001')
print(list(sttok.tokenize(u'我家没有电脑。')))



['我家', '没有', '电脑', '。']


In [63]:
jieba_counts=defaultdict(int)
stanford_counts=defaultdict(int)
lengthsum=0
for n,s in enumerate(sentences):
    lengthsum+=len(s)-1
    fullsentence="".join(s)
    jieba_cut=jieba.lcut(fullsentence)
    stanford_cut=list(sttok.tokenize(fullsentence))
    jieba_counts[segmentation_distance(s,jieba_cut)]+=1
    stanford_counts[segmentation_distance(s,stanford_cut)]+=1
    #if not n%100:
    #    print(n)

In [64]:
print('average words in a sentence: ',lengthsum/(n+1))
print('average jieba distance: ',sum([k*v for k,v in jieba_counts.items()])/sum(jieba_counts.values()))
print('average stanford distance: ',sum([k*v for k,v in stanford_counts.items()])/sum(stanford_counts.values()))
print("distance: jieba    stanford")
for k in range(101):
    print(k,' :     ',jieba_counts[k],'    ',stanford_counts[k])


average words in a sentence:  26.309313260288153
average jieba distance:  4.380726425675093
average stanford distance:  3.4891294970235527
distance: jieba    stanford
0  :      4622      4843
1  :      7516      8265
2  :      7527      8466
3  :      5991      7102
4  :      4796      5329
5  :      3683      3767
6  :      2812      2593
7  :      2062      1844
8  :      1638      1273
9  :      1283      827
10  :      916      577
11  :      690      423
12  :      577      274
13  :      440      199
14  :      344      163
15  :      250      100
16  :      207      75
17  :      180      56
18  :      147      43
19  :      97      25
20  :      90      19
21  :      83      14
22  :      63      6
23  :      46      6
24  :      49      6
25  :      39      9
26  :      19      6
27  :      25      6
28  :      11      1
29  :      22      0
30  :      14      2
31  :      12      0
32  :      10      1
33  :      11      1
34  :      13      0
35  :      3      2
36  :      3

In [65]:
a=['我们', '变', '而', '以', '书会友', '，', '以', '书', '结缘', '，', '把', '欧美']

In [66]:
b=jieba.lcut(''.join(a))
c=list(sttok.tokenize(''.join(a)))

In [67]:
print(a)
print(b)
print(c)

['我们', '变', '而', '以', '书会友', '，', '以', '书', '结缘', '，', '把', '欧美']
['我们', '变而', '以书', '会友', '，', '以书', '结缘', '，', '把', '欧美']
['我们', '变', '而', '以', '书', '会友', '，', '以', '书', '结缘', '，', '把', '欧美']


In [68]:
segmentation_distance(a,b)

3

In [69]:
segmentation_distance(a,c)

1